Script For MarkovChain Models and getting the steady state distribuution of Boolean Perturbed Networks
- written by Ioanna Deni 2018

This script uses networks downloaded as sbml files from website https://cellcollective.org/# and analysed their properties by using BoolNet in r 3.4.4 (https://cran.r-project.org/web/packages/BoolNet/BoolNet.pdf)

In [ ]:
#install.packages("BoolNet", repos = "http://cran.us.r-project.org")
library(BoolNet)
library(markovchain)

Simulating the network and examining the steady state distribution of transitions between states due to perturbations will reveal the probability of a state to develop into an alternative state or to maintain itself. 

In [ ]:
finalNetwork <- loadSBML("CellCycleTranscriptionbyCoupledCDKandNetwork.sbml")

sim <- markovSimulation(finalNetwork)
# The simulation shows the states that are reached after 1000 iterations
transitionProb <- getTransitionProbabilities(sim)
#retrieving the adjacency transition matrix 

xNet <- vector()
statesNet <- list()

#IMPORTANT: the following loop needs to change for the number of genes of the network -> this example has 9 genes

for (rows in 0:nrow(transitionProb)){
  #this will be a string of zeros describing the initial state  
  lineInitNet <- paste0(transitionProb[rows, 1], transitionProb[rows, 2], transitionProb[rows, 3], transitionProb[rows, 4], transitionProb[rows, 5], transitionProb[rows, 6], transitionProb[rows, 7], transitionProb[rows, 8], transitionProb[rows, 9])
  #this will be a string of zeros describing the final state after 1000 iterations
  lineFinalNet <- paste0(transitionProb[rows, 10], transitionProb[rows, 11], transitionProb[rows, 12], transitionProb[rows, 13], transitionProb[rows, 14], transitionProb[rows, 15], transitionProb[rows, 16], transitionProb[rows, 17], transitionProb[rows, 18])
  probN <- transitionProb[rows, 19] 
    
  xNet <- append(xNet, lineInitNet) 
  statesNet <- append(statesNet, lineInitNet)  
  xNet <- append(xNet, lineFinalNet) 
  xNet <- append(xNet, probN)   
}

#converting the vector in numbers 
nNet <- factor(xNet)

#creating a matrix to represent the transition matrix
myMatrixSNet <- matrix(nNet,ncol=3,byrow=TRUE)
#naming the columns appropriately
myDataSNet <- matrix(0, nrow = nrow(transitionProb), ncol= nrow(transitionProb), dimnames = list(statesNet, statesNet))
#and filling it with the values
for (i in 1:nrow(myDataSNet)){    
    for (j in 1:ncol(myDataSNet)){
        #filling it with ones when there is a transition and zero where there is no transition
        if (colnames(myDataSNet)[j] %in% myMatrixSNet[i, 2]){
            myDataSNet[i, j] <- 1
        }
    }  
}

#getting the transpose so in myMatrix the col sums are 1 
myMatrix <- t(myDataSNet)

The perturbations are defined by flipping a state from 0 to 1 or from 1 to zero at the initial state and following that new perturbed state after 1000 iterations to see at what basin it reached using the same adjecency matrix. The single perturbation is done for all genes found in each state and the resullting probability is a fraction (# perturbed state reached state basin n)/(# of genes) for all basin found in the network. Further, there can be two flippings of a state, two genes switching from 0 to 1 or from 1 to 0, and there can be three flippings and four and n for n genes. Thus this following algorithm assigns the probability of 1....n flippings based on the binomial coefficient while assuming that the probability of flipping is very small 0.01  

In [ ]:
probMatrix <- matrix(0, nrow = nrow(myMatrix), ncol= nrow(myMatrix), dimnames = list(statesNet, statesNet))
# matrix to save the probability of transition matrix
e <- 0.01 #this just needs to be a small number - probability of flipping 
m <- 9 #number of bits of a state - number of genes on the network

for (col in 1:ncol(myMatrix)){    
    for (row in 1:nrow(myMatrix)){ 
        currentState <- colnames(myMatrix)[col]
        nextState <- rownames(myMatrix)[row]
        order <- 0 # this will store the number representing the different states of the initial to the perturbed state 
        for (char in 1:nchar(currentState)){
            characterNow <- substr(currentState, char, char)
            characterNext <- substr(nextState, char, char)
            if (characterNow != characterNext){
                order <- order + 1
            }   
        } 
        if (order == 0){
            probMatrix[row, col] <- 0
        }else {
            probMatrix[row, col] <- (e^(order)) * ((1-e)^(m-order)) #here is the assignment of the binomial probability distribution 
        }
    }
}

The myMatrix has the distinct transition matrix and teh probMatrix has the perturbed transition states. In the followin part these two matrices are combined.

In [ ]:
combMatrix <- matrix(0, nrow = nrow(myMatrix), ncol= nrow(myMatrix), dimnames = list(statesNet, statesNet))
for (col in 1:ncol(myMatrix)){ 
    for (row in 1:nrow(myMatrix)){ 
        combMatrix[row, col] <- ((1-e)^m)*myMatrix[row, col] + probMatrix[row, col]}}

Finaly using the seconnd library markovchain the steady state distribution of all states can be calculated.

In [ ]:
mcNewMatrix <- as(combMatrix, "markovchain")
probMcCanonic <- canonicForm(t(mcNewMatrix)) # thsi reduces the running time
myStates <- steadyStates(probMcCanonic)

myStates #this will add up to 1